## 텍스트와 바이트 
- 문자, 코드 포인트, 바이트 표현 
- 이진 시퀀스 특징
- 유니코드 및 문자셋 코덱
- 인코딩 에러 헨들링
- 텍스트 파일 다루기
- 인코딩 및 표준 입출력
- 정규화 텍스트 비교 
- 정규화, 케이스 폴딩, 기호 제거를 위한 유틸
- 유니코드 텍스트 정렬,
- 문자 메타데이터
- str, bytes 이중모드 API

# 4.1 문자 문제 
python3 str 은 유니코드문자를 가져옴. 문자 단위 원소 및 특정 바이트 표현을 구분
- 문자 code point : 10진수 0 - 1,114,111 ㄱ까지의 숫자 , 유니코드 표준에서 U+접두사를 붙여 4자리, 6자리의 16진수로 표현
- 문자를 표현하는 실제 바이트는 이코딩에 따라 달라짐, A (U+0041) --> UTF-8 \x41, UTF-16 \x41\x00 으로 인코딩됨


In [1]:
#바이트 표현  b로 시작
b = b'caf\xc3\xa9'
print(b)

b'caf\xc3\xa9'


In [2]:
s = b.decode('utf8')
s #유니코드 문자열 

'café'

In [3]:
len(s)

4

In [4]:
b = s.encode('utf8')
b, len(b)

(b'caf\xc3\xa9', 5)

# 4.2 바이트 기본지식
- 이진 시퀀스를 위한 내장 자료형은 bytes, bytearray 두가지가 있음
- 0, 225 사이의 정수가 들어가 있으며 슬라이싱하면 같은 자료형이 생성

In [6]:
cafe = bytes('café', encoding='utf_8')
cafe, cafe[0], cafe[:2]

(b'caf\xc3\xa9', 99, b'ca')

In [7]:
cafe_arr = bytearray(cafe)
cafe_arr, cafe_arr[-1:]

(bytearray(b'caf\xc3\xa9'), bytearray(b'\xa9'))

In [8]:
#s[0] == s[:1]이 되는 시퀀스형은 str이 유일하다. 
# 그외 모든 시퀀스의 경우, s[i]는 항목 하나를, s[i:i+1]은 안에 s[i] 항목을 가진 동일한 자료형의 시퀀스를 반환한다.
test = str('abcdefg')
test[0], test[:1]

('a', 'a')

In [9]:
test = list('abcdefg')
test[0], test[:1]

('a', ['a'])

- 이진 시퀀스는 정수형의 시퀀스이긴 하지만, 실제로는 아스키 텍스트가 들어가는 경우가 많다. 
- 따라서 각 바이트 값에 따라 다음과 같이 세 가지 형태로 출력된다.
1. 화면에 출력 가능한 아스키 문자(공백에서 ~까지)는 아스키 문자 그대로 출력
2. 탭, 개행 문자, 캐리지 리턴, 백슬레시()는 이스케이프 시퀀스(\t, \n, \r, \)로 출력
3. 그외의 값은 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력
- bytes와 bytearray는 포매팅하는 format(), format_map() 메서드를 제외하고는 str이 제공하는 메서드를 모두 지원
- casefold(), isdecimal(), isidentifier(), isnumeric (), isprintable(), encode() 등 유니코드 데이터에 관련한 메서드도 지원한다. 
- 이진 시퀀스는 fromhex()라는 str에 없는 클래스 메서드를 제공하여 공백으로 구분된 16진수 쌍을 파싱해서 이진 시퀀스를 만들 수 있다.

In [13]:
test = bytes.fromhex('31 4B CE A9')
test, test.decode('utf8')

(b'1K\xce\xa9', '1KΩ')

In [14]:
bytes(6), bytes([6])

(b'\x00\x00\x00\x00\x00\x00', b'\x06')

In [17]:
import array
numbers = array.array('h', [-2,-1,0,1,2]) 
# 'h' 타입 코드는 short int(16비트) 형의 배열을 생성한다.
numbers, bytes(numbers)

(array('h', [-2, -1, 0, 1, 2]), b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00')

# 4.2.1 구조체와 메모리 뷰 
- struct : 패킹된 바이트 --> 튜플, 튜플 --> 패킹된 바이트

In [19]:
import struct 
fmt = '<3s3sHH' # struct 포맷을 지정. <는 리틀엔디언, 3s3s는 3바이트 시퀀스, HH는 16비트 정수 두개를 나타냄
                # 리틀 엔디안 방식은 낮은 주소에 데이터의 낮은 바이트(LSB, Least Significant Bit)부터 저장하는 방식
with open('image.png', 'rb') as fp:
    img = memoryview(fp.read()) # memoryview는 외부자료를 버퍼에 저장 , bytes, bytearray 형식 지원 

In [21]:
header = img[:10]
header


In [22]:
bytes(header)


b'\x89PNG\r\n\x1a\n\x00\x00'

In [23]:
struct.unpack(fmt, header) # 종류, 버전, 너비, 높이 튜플로 언패킹

(b'\x89PN', b'G\r\n', 2586, 0)

# 4.3 기본 인코더/디코더
- 파이썬에는 텍스트-바이트 변환을 위한 100여개의 코덱이 포함

In [24]:
for codec in ['latin_1', 'utf8', 'utf16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf8	b'El Ni\xc3\xb1o'
utf16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


In [25]:
for codec in ['ascii','latin_1','cp1252','utf8','utf16','utf-16le']:
    print(codec, 'A'.encode(codec), sep='\t')

ascii	b'A'
latin_1	b'A'
cp1252	b'A'
utf8	b'A'
utf16	b'\xff\xfeA\x00'
utf-16le	b'A\x00'


- latin1(iso8859_1) : 다른 인코딩 및 유니코드의 기반이 되는 중요한 인코딩
- cp1252 : MS에서 둥근 따옴표 및 유로화 기호 등을 추가해서 latin1을 확장
- cp437 : 상자를 그리기 위한 문자를 포함해서 원래 IBM PC에서 사용하는 문자셋
- gb2312 : 중국 본토에서 사용하는 간체를 인코딩하기 위한 레거시 표준
- utf-8 : 웹에서 8비트 인코딩을 하기 위해 가장 널리 사용되는 인코딩 방식
- utf-16le : 16비트 인코딩 체계인 UTF-16의 한 형태


# 4.4 인코딩/디코딩 문제 이해하기
- UnicodeError : 범용에러
- UnicodeEncodeError : str을 이진 시퀀스로 변환할 때
- UnicodeDecodeError : 이진 시퀀스를 str로 읽어 들일 때
- SyntaxError : 소스 코드가 예기치 않은 방식으로 인코딩되어 있는 경우

## 4.4.1 UnicodeEncodeError 처리하기
- 대부분의 비UTF 코덱은 유니코드 문자의 일부만 처리 가능  
- 텍스트를 바이트로 변환할 때 문자가 대상 인코딩에 정의되어 있지 않으면 UnicodeEncodeError가 발생

In [27]:
city = 'São Paulo'
city.encode('utf-8')

b'S\xc3\xa3o Paulo'

In [28]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [29]:
city.encode('iso8859_1')

b'S\xe3o Paulo'

In [30]:
city.encode('cp437') #오류발생

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [31]:
city.encode('cp437', errors='ignore') # 인코딩할 수 없는 문자를 건너뛴다. 좋지 않다.

b'So Paulo'

In [32]:
city.encode('cp437', errors='replace') # 물음표(?)로 치환한다.

b'S?o Paulo'

In [33]:
city.encode('cp437', errors='xmlcharrefreplace') # XML 객체로 치환한다.

b'S&#227;o Paulo'

## 4.4.2 UnicodeDecodeError 처리하기
- 이진 시퀀스를 텍스트로 변환할 때 정당한 문자로 변환할 수 없으면 UnicodeDecodeError가 발생 
- 'cp1252' 등 많은 레거시 8비트 코덱은 에러를 발생시키지 않고 바이트 스트림으로 디코딩 가능

In [34]:
octets = b'Montr\xe9al' # latin1 으로 인코딩
octets.decode('cp1252') # cp1252는 latin1의 슈퍼셋이므로 제대로 디코딩됨

'Montréal'

In [36]:
octets.decode('iso8859_7') # 그리스 문자를 위한 코덱이므로 엉뚱하게 해석함

'Montrιal'

In [37]:
octets.decode('koi8_r') # 러시아어를 위한 코덱

'MontrИal'

In [39]:
octets.decode('utf8') # 에러발생

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [40]:
octets.decode('utf8', errors='replace') # �는 알 수 없는 문자를 표현하기 위해 사용하는 유니코드 공식 치환 문자

'Montr�al'

In [41]:
'é'.encode('utf8')

b'\xc3\xa9'

## 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError
- 파이썬 3부터는 UTF-8을 소스 코드 기본 인코딩 방식으로 사용
- 인코딩 선언 없이 비UTF-8로 인코딩된 .py 모듈을 로딩하면 에러가 발생 
- 파일 꼭대기에 coding 주석을 달아서 해결 가능

In [42]:
# coding: cp1252

print('Olá Mundo!')

Olá Mundo!


## 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법
- 바이트 시퀀스의 인코딩 방식을 알아내는 방법은 없음 
- 별도로 인코딩 정보를 가져와야 함 
- 경험과 통계를 통해 추정가능 --> Chardet 패키지

In [44]:
! chardetect test.txt

test.txt: ascii with confidence 1.0


## 4.4.5 BOM: 유용한 깨진 문자 
- 인코딩된 텍스트의 이진 시퀀스는 인코딩에 대한 정보를 명시적으로 전달하지 않음 
- UTF 포맷은 텍스트 앞에 바이트 순서 표시(byte order mark, BOM)을 추가할 수 있다.

In [45]:
u16 = 'El Niño'.encode('utf16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

In [58]:
list(u16)

[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

In [46]:
# b'\xff\xfe' 문자가 바로 바이트 순서 표시 -->  인코딩한 인텔 CPU의 '리틀엔디언' 바이트 순서를 나타낸다.

# 4.5 텍스트 파일 다루기
- 텍스트를 처리할 때는 최대한 bytes-str 변환 속도 중요
- 파이썬은 open() 함수를 이용하여 텍스트를 읽고 쓸 때 모든 인코딩/디코딩 작업을 수행 
- my_file.read()에서 str 객체를 가져와서 처리하고 my_file.write()에 전달
- 기본 인코딩에 의존하다보면 문제가 생길 때가 있다.

In [47]:
open('cafe.txt', 'w', encoding='utf8').write('café')

4

In [48]:
open('cafe.txt').read() # 이렇게 읽을 때 인코딩을 지정하지 않으면 오류가 발생할 수도 있음

'café'

In [49]:
# encoding 지정
fp = open('cafe.txt', 'w', encoding='utf8')
fp

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf8'>

In [50]:
fp.write('café') # TetxtIOWrapper 객체의 write() 메서드는 저장한 유니코드 문자수를 반환

4

In [51]:
fp.close()

In [52]:
import os
os.stat('cafe.txt').st_size # utf8 에서는 é가 2개의 바이트가 되기 떄문

5

In [53]:
fp2 = open('cafe.txt', encoding='cp1252')
fp2

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp1252'>

In [54]:
fp2.read()

'cafÃ©'

In [55]:
fp3 = open('cafe.txt', 'rb')
fp3

<_io.BufferedReader name='cafe.txt'>

In [56]:
fp3.read()

b'caf\xc3\xa9'

In [57]:
# 인코딩 방식을 알아내기 위해 파일 내용을 분석하는 경우가 아니라면 텍스트 파일을 이진 모드로 열지 않는 것이 좋다. 
# 인코딩 방식을 알아낼 때도 Chardet 모듈을 사용하는 것이 좋다.


## 4.5.1 기본 인코딩 설정: 정신 나간 거 아냐? 

In [59]:
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

# locale.getpreferredencoding() 함수가 반환하는 설정이 기본 인코딩 설정

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


# 4.6 제대로 비교하기 위해 유니코드 정규화하기¶
- 유니코드에는 결합 문자(발음 구별 기호 등)가 있기 때문에 문자열 비교가 간단하지 않다.

In [60]:
s1 = 'café'
s2 = 'cafe\u0301' # U+0301 은 `COMBINING ACUTE ACCENT`
s1, s2

('café', 'café')

In [61]:
len(s1), len(s2), s1==s2

(4, 5, False)

In [62]:
# NFC(Normalization Form C) : 코드 포인트를 조합해서 가장 짧은 동일 문자열을 생성
# NFD(Normalization Form D) : 조합된 문자를 기본 문자와 별도의 결합 문자로 분리
# 사용하가 입력하는 텍스트는 기본적으로 NFC 형태
from unicodedata import normalize
len(normalize('NFC',s1)), len(normalize('NFC',s2)), len(normalize('NFD',s1)), len(normalize('NFD',s2))

(4, 4, 5, 5)

In [63]:
normalize('NFC',s1), normalize('NFC',s2), normalize('NFC',s1)==normalize('NFC',s2)

('café', 'café', True)

In [64]:
normalize('NFD',s1), normalize('NFD',s2), normalize('NFD',s1)==normalize('NFD',s2)

('café', 'café', True)

In [65]:
from unicodedata import normalize, name
ohm = '\u2126'
name(ohm)

'OHM SIGN'

In [66]:
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [67]:
ohm, ohm_c, ohm == ohm_c

('Ω', 'Ω', False)

In [68]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

In [4]:
from unicodedata import normalize, name
half = '½'
normalize('NFKC', half), normalize('NFKD', half)

('1⁄2', '1⁄2')

In [70]:
four_squared = '4²'
normalize('NFKC', four_squared), normalize('NFKD', four_squared)

('42', '42')

In [71]:
micro = 'μ'
micro_kc = normalize('NFKC', micro)
micro, micro_kc

('μ', 'μ')

In [72]:
ord(micro), ord(micro_kc)

(956, 956)

## 4.6.1 케이스폴딩
- 텍스트를 소문자로 변환하는 연산 
- str.casefold() 메서드를 이용

In [5]:
eszett = 'ß'
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [6]:
eszett_cf = eszett.casefold()
eszett_cf2 = eszett.lower()
eszett, eszett_cf, eszett_cf2

('ß', 'ss', 'ß')

## 4.6.2 정규화된 텍스트 매칭을 위한 유틸리티 함수

In [11]:
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() == normalize('NFC', str2).casefold())

# 대소문자 구분, NFC 사용
s1 = 'café'
s2 = 'cafe\u0301' # U+0301 은 `COMBINING ACUTE ACCENT`
s1 == s2, nfc_equal(s1, s2), fold_equal(s1, s2), nfc_equal('A', 'a')

(False, True, True, False)

In [12]:
s3 = 'Straße'
s4 = 'strasse'
s3 == s4, nfc_equal(s3, s4), fold_equal(s3, s4), fold_equal('A', 'a')

(False, False, True, True)

In [15]:
import unicodedata
import string


def shave_marks(txt):
    """Remove all diacritic marks"""
    norm_txt = unicodedata.normalize('NFD', txt)  # 정규화 실행
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))  # 발음 기호가 아닌 경우만 결함
    return unicodedata.normalize('NFC', shaved)  # 

def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # <2>
            continue  # ignore diacritic on Latin base char
        keepers.append(c)                             # <3>
        # if it isn't combining char, it's a new base char
        if not unicodedata.combining(c):              # <4>
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)   # <5>


single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  # <1>
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({  # <2>
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  # <3>


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  # <4>


def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))     # <5>
    no_marks = no_marks.replace('ß', 'ss')          # <6>
    return unicodedata.normalize('NFKC', no_marks)  # <7>

# Radical folding and text sanitizing.
# Handling a string with `cp1252` symbols:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
print(shave_marks(order))
print(shave_marks_latin(order))
print(dewinize(order))
print(asciize(order))

“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”
“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”
"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."
"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."


In [16]:
greek = 'Ζέφυρος, Zéfiro'
print(shave_marks(greek))
print(shave_marks_latin(greek))
print(dewinize(greek))
print(asciize(greek))

Ζεφυρος, Zefiro
Ζέφυρος, Zefiro
Ζέφυρος, Zéfiro
Ζέφυρος, Zefiro


# 4.7 유니코드 텍스트 정렬하기

In [17]:
import locale
locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8') #현지어 설정

'pt_BR.UTF-8'

In [19]:
# 현지어 설정에 따라 정렬
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=locale.strxfrm)
sorted_fruits, sorted(fruits)

(['açaí', 'acerola', 'atemoia', 'cajá', 'caju'],
 ['acerola', 'atemoia', 'açaí', 'caju', 'cajá'])

# 4.8 유니코드 데이터베이스
- 코드포인트를 문자열로 맵핑하는 테이블, 메타데이터, 연관 방법 등을 담고있음
- 문자 출력 가능여부, 문자, 십진수, 수치형 기호 구분

In [20]:
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print(f'U+{ord(char):04x}',                       # <1>
          char.center(6),                             # <2>
          're_dig' if re_digit.match(char) else '-',  # <3>
          'isdig' if char.isdigit() else '-',         # <4>
          'isnum' if char.isnumeric() else '-',       # <5>
          f'{unicodedata.numeric(char):5.2f}',        # <6>
          unicodedata.name(char),                     # <7>
          sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


## 4.9.1 정규표현식 str, bytes
- bytes 정규표현식으 \d, \w 같은 패턴은 아스키 문자만 매칭
- str로 패턴을 만들면 유니코드 문자도 매칭 

In [21]:
import re

re_numbers_str = re.compile(r'\d+')     # str 형 정규표현식
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')  # bytes 형 정규표현식
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"  # <3>
            " as 1729 = 1³ + 12³ = 9³ + 10³.")        # <4>

text_bytes = text_str.encode('utf_8')  # <5>

print(f'Text\n  {text_str!r}')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))      # <6>
print('  bytes:', re_numbers_bytes.findall(text_bytes))  # <7>
print('Words')
print('  str  :', re_words_str.findall(text_str))        # <8>
print('  bytes:', re_words_bytes.findall(text_bytes))    # <9>

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


## 4.9.2 os 모듈 함수에서 str과 bytes
- 경로를 받는 모든 os 모듈 함수는 str 혹은 bytes로 arg를 받음 

In [22]:
os.listdir('.')

['ch4_textbyte.ipynb',
 'cafe.txt',
 '.gitignore',
 'ch5_firstclass_function.ipynb',
 'test.txt',
 '.git',
 'image.png']

In [23]:
os.listdir(b'.') #utf-8로 인코딩한 결과

[b'ch4_textbyte.ipynb',
 b'cafe.txt',
 b'.gitignore',
 b'ch5_firstclass_function.ipynb',
 b'test.txt',
 b'.git',
 b'image.png']